In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import numpy as np
from sklearn.decomposition import PCA

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
db_catboost = DFDB('../trial/catboost.pkl', auto_commit=False)
df_trial_catboost = db_catboost.select()
df_trial_catboost['kfold'] = df_trial_catboost['param'].apply(lambda x: x['kfold'])

In [7]:
db_xgbm = DFDB('../trial/xgbm.pkl', auto_commit=False)
df_trial_xgbm = db_xgbm.select()
df_trial_xgbm['kfold'] = df_trial_xgbm['param'].apply(lambda x: x['kfold'])

In [8]:
db_lgbm = DFDB('../trial/lgbm.pkl', auto_commit=False)
df_trial_lgbm = db_lgbm.select()
df_trial_lgbm['kfold'] = df_trial_lgbm['param'].apply(lambda x: x['kfold'])

In [9]:
db_randomforest = DFDB('../trial/randomforest.pkl', auto_commit=False)
df_trial_randomforest = db_randomforest.select()
df_trial_randomforest['kfold'] = df_trial_lgbm['param'].apply(lambda x: x['kfold'])

In [10]:
db_extratrees = DFDB('../trial/extratrees.pkl', auto_commit=False)
df_trial_extratrees = db_extratrees.select()
df_trial_extratrees['kfold'] = df_trial_extratrees['param'].apply(lambda x: x['kfold'])

In [11]:
db_gradientboosting = DFDB('../trial/gradientboosting.pkl', auto_commit=False)
df_trial_gradientboosting = db_gradientboosting.select()
df_trial_gradientboosting['kfold'] = df_trial_gradientboosting['param'].apply(lambda x: x['kfold'])

In [12]:
db_knn = DFDB('../trial/knn.pkl', auto_commit=False)
df_trial_knn = db_knn.select()
df_trial_knn['kfold'] = df_trial_knn['param'].apply(lambda x: x['kfold'])

In [13]:
db_svr = DFDB('../trial/svr.pkl', auto_commit=False)
df_trial_svr = db_svr.select()
df_trial_svr['kfold'] = df_trial_knn['param'].apply(lambda x: x['kfold'])

In [14]:
db_fm = DFDB('../trial/fm.pkl', auto_commit=False)
df_trial_fm = db_fm.select()
df_trial_fm['kfold'] = df_trial_knn['param'].apply(lambda x: x['kfold'])

In [15]:
db_lasso = DFDB('../trial/lasso.pkl', auto_commit=False)
df_trial_lasso = db_lasso.select()
df_trial_lasso['kfold'] = df_trial_lasso['param'].apply(lambda x: x['kfold'])

In [16]:
db_ridge = DFDB('../trial/ridge.pkl', auto_commit=False)
df_trial_ridge = db_ridge.select()
df_trial_ridge['kfold'] = df_trial_ridge['param'].apply(lambda x: x['kfold'])

In [17]:
db_kerasmlp = DFDB('../trial/kerasmlp.pkl', auto_commit=False)
df_trial_mlp = db_kerasmlp.select()
df_trial_mlp['kfold'] = df_trial_mlp['param'].apply(lambda x: x['kfold'])

In [18]:
db_frgf = DFDB('../trial/frgf.pkl', auto_commit=False)
df_trial_frgf = db_frgf.select()
df_trial_frgf['kfold'] = df_trial_frgf['param'].apply(lambda x: x['kfold'])

In [19]:
db_stacknet_pca = DFDB('../trial/stacknet_pca.pkl', auto_commit=False)
df_trial_stacknet_pca = db_stacknet_pca.select()
df_trial_stacknet_pca['kfold'] = df_trial_frgf['param'].apply(lambda x: x['kfold'])

In [20]:
trial_lst = [
    df_trial_catboost.loc[452], df_trial_xgbm.loc[1172], df_trial_lgbm.loc[2156],df_trial_gradientboosting.loc[306],
             df_trial_randomforest.loc[297],df_trial_extratrees.loc[459],
            df_trial_knn.loc[17], df_trial_svr.loc[7],df_trial_fm.loc[313],
            df_trial_lasso.loc[8], df_trial_ridge.loc[15],
        df_trial_mlp.loc[13], df_trial_frgf.loc[200],
            ]
name_lst = [
    'cb452', 'xgbm1172', 'lgbm2156',
    'gbm306','rf297','et459',
    'knn17','svr7', 'fm313',
    'lasso8','ridge15',
    'mlp13','frgf200'
]

In [90]:
for r in trial_lst:
    print(r['param']['algorithm']['cls'], r['param']['algorithm']['init'], r['param']['kfold']['type'], r['param']['kfold']['n_splits'], len(r['param']['columns']), r['val_mae'], r['mae_diff'])
    print()

cb.CatBoostRegressor {'num_trees': 929, 'depth': 5, 'learning_rate': 0.01571023413441655, 'l2_leaf_reg': 0.49142786214498846, 'bagging_temperature': 0.6008299770689625, 'random_strength': 0.7930166632962454, 'random_state': 1569} stratified 3 30 1.9005345457826346 0.04705627937966006

xgb.XGBRegressor {'max_depth': 3, 'max_bin': 77, 'eta': 0.28071904276671816, 'colsample_bytree': 0.9102857670808742, 'min_child_weight': 271, 'n_estimators': 113, 'subsample': 0.6831182550379268, 'reg_lambda': 0.812561328581548, 'reg_alpha': 0.007362369490410914, 'n_jobs': 16} stratified 3 25 1.9655602276971649 0.040781720685815426

lgb.LGBMRegressor {'learning_rate': 0.19329183957823715, 'feature_fraction': 0.7363782645537933, 'bagging_fraction': 0.8585309819625903, 'min_data_in_leaf': 351, 'lambda_l1': 95.63411922439124, 'lambda_l2': 48.949784919880365, 'max_bin': 25, 'num_leaves': 8, 'random_state': 5040, 'n_jobs': 32} stratified 3 25 1.9528967095257748 0.04172569264886117

GradientBoostingRegressor {'

In [21]:
df_train_oof = pd.DataFrame()
df_test_oof = pd.DataFrame()
for df_, name_ in zip(trial_lst, name_lst):
    try:
        df_test_oof_i = df_['df_test_pred']
        df_test_oof[name_] = np.mean(df_test_oof_i.drop(columns=['index']).values, axis=1)
        
        df_train_oof_i = df_['df_valid_pred']
        df_train_oof[name_] = df_train_oof_i['predict']
        
    except Exception as e:
        print(name_, e.__str__())

df_train_oof['index'] = df_train_oof_i['index']
df_test_oof['index'] = df_test_oof_i['index']

In [22]:
df_train = pd.merge(df_train, df_train_oof, on='index')
df_test = pd.merge(df_test, df_test_oof, on='index')

In [23]:
df_pca1_train = pd.read_pickle('../feats/pca.pkl')
df_pca1_test = pd.read_pickle('../feats/pca_test.pkl')

In [24]:
df_train = pd.concat([df_train, df_pca1_train], axis=1)
df_test = pd.concat([df_test, df_pca1_test], axis=1)

In [99]:
df_spec_train = pd.read_pickle('../feats/spec_features.pkl')
df_spec_test = pd.read_pickle('../feats/spec_features_test.pkl')

In [26]:
df_train = pd.merge(df_train, df_spec_train, on='index')
df_test = pd.merge(df_test, df_spec_test, on='index')

In [27]:
df_tsne_train = pd.read_pickle('../feats/tsne.pkl')
df_tsne_test = pd.read_pickle('../feats/tsne_test.pkl')

In [28]:
df_train = pd.merge(df_train, df_tsne_train, on='index')
df_test = pd.merge(df_test, df_tsne_test, on='index')

In [29]:
df_vae_train = pd.read_pickle('../feats/vae.pkl')
df_vae_test = pd.read_pickle('../feats/vae_test.pkl')

In [30]:
df_train = pd.merge(df_train, df_vae_train, on='index')
df_test = pd.merge(df_test, df_vae_test, on='index')

In [29]:
# mytrial =[]
# columns= param['columns']
# df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train[columns+['y','index','group','label']], param, df_test=df_test[columns+['index']], trial=mytrial, is_output_feature_importance=False, remark='rebuild mystacknet_pca_469')

In [57]:
pd.DataFrame(mytrial)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-29 12:35:32.727234,47,1.730152,0.000015,1.778465,0.000249,0.048313


In [31]:
param = {'columns': ['cb452',
  'xgbm1172',
  'lgbm2156',
  'gbm306',
  'rf297',
  'et459',
  'knn17',
  'svr7',
  'fm313',
  'lasso8',
  'ridge15',
  'mlp13',
  'frgf200',
  'spkt_welch_density__coeff_3',
  'abs_q25_5',
  'q25_roll_std_100',
  'abs_q75_6',
  'abs_q75_7',
  'spkt_welch_densitycoeff_2',
  'abs_q01_4',
  'iqr_6',
  'q05_roll_std_100',
  'q05_roll_std_1000',
  'median__roll_std',
  'abs_q01_5',
  "number_peaks{'n': 10}",
  'FFT_Mag_75q0',
  "value_count{'value': 1}",
  'q01_roll_std_100',
  'abs_q95_2',
  'abs_q95_6',
  'MA_1000MA_std_mean_7',
  'q05_roll_std_10',
  'q01_roll_std_1000',
  'abs_max_roll_mean_1000',
  'abs_q75_2',
  'abs_q05_6',
  '5000std_quantile25',
  "number_crossing_m{'m': 1}",
  "autocorrelation{'lag': 5}",
  'q75_roll_std_10',
  'q05_2',
  '5000smoothness_quantile05',
  '5000variance_median__pca6',
  'abs_q05_6_pca1',
  '5000std_quantile75_pca4',
  "range_count{'min': 0, 'max': 1000000000000.0}_pca1"],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'num_trees': 397,
   'depth': 6,
   'learning_rate': 0.04568702913200931,
   'l2_leaf_reg': 42.67420651153853,
   'bagging_temperature': 0.8288030970057758,
   'random_strength': 0.648173282450518,
   'random_state': 9734,
   'logging_level': 'Silent'},
  'fit': {}}}

In [32]:
mfcc_columns = ['mfcc_5_abs_mean',
 'mfcc_13_mean',
 'mfcc_accelerate_8_variance',
 'mfcc_accelerate_1_kurtosis',
 'mfcc_13_quantile25',
 'mfcc_5_mean']

In [33]:
lgbm_top200 = ['spkt_welch_densitycoeff_2',
 'spkt_welch_density__coeff_3',
 'q25_roll_std_100',
 '3th_peak_freq',
 'min_roll_std_100',
 'q05_roll_std_100',
 'iqr_6',
 'abs_max_8',
 'mfcc_5_abs_mean',
 "number_peaks{'n': 10}",
 'ave10_7',
 'mfcc_13_mean',
 "autocorrelation{'lag': 5}",
 'mfcc_accelerate_8_variance',
 'abs_max_7',
 'mfcc_accelerate_1_kurtosis',
 'q05_roll_std_1000',
 'spkt_welch_density__coeff_42',
 'max_to_min_diff_5',
 'mfcc_13_quantile25',
 'fft_coefficientcoeff_80__attr_"imag"',
 'abs_q25_5',
 'mfcc_5_mean',
 'median__roll_std',
 '5000skewness_max_',
 'fft_coefficientcoeff_6__attr_"abs"',
 'partial_autocorrelationlag_5',
 'abs_min_8',
 'spkt_welch_density__coeff_28',
 'ar_coefficientk_10__coeff_3',
 'abs_q75_7',
 'mfcc_accelerate_15_min',
 'abs_max_4',
 'mfcc_10_quantile25',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'spkt_welch_density__coeff_79',
 "value_count{'value': 1}",
 '5000peak_peak_amp_max_',
 'min__roll_std',
 'mfcc_4_rolling_std_mean',
 'q01_roll_std_1000',
 'mfcc_3_abs_q75',
 'mfcc_5_quantile95',
 'fft_coefficientcoeff_16__attr_"imag"',
 'abs_q01_4',
 'mfcc_6_quantile05',
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 '5000quantile25skewness_',
 'mfcc_0_abs_q25',
 'mfcc_delta_3_quantile01',
 'spkt_welch_density__coeff_99',
 'mfcc_9_mean',
 'mfcc_12_mean',
 'q75_roll_mean_10',
 'max_to_min',
 'mfcc_3_abs_mean',
 'kurt_7',
 'mfcc_14_quantile01',
 'mfcc_10_mean',
 'mfcc_delta_13_min',
 'mfcc_delta_13_abs_max',
 '5000no_zero_crossing_mean_',
 'abs_q75_6',
 'mfcc_12_quantile99',
 'mfcc_accelerate_6_skewness',
 'mfcc_5_max',
 '5000smoothness_entropy_',
 '5000median_skewness_',
 'mfcc_accelerate_8_min',
 'abs_min_3',
 '5000quantile75mean_',
 "number_crossing_m{'m': 1}",
 '5000smoothness_std_',
 'mfcc_9_abs_q25',
 'flac3_1_quantile99',
 'mfcc_5_skewness',
 'fft_coefficientcoeff_56__attr_"angle"',
 'fft_coefficientcoeff_70__attr_"abs"',
 'mfcc_10_abs_q75',
 'fft_coefficientcoeff_24__attr_"angle"',
 'med_7',
 'spkt_welch_density__coeff_73',
 'abs_q99_8',
 'ave10_6',
 'spkt_welch_density__coeff_38',
 'skew_1',
 'mfcc_delta_3_abs_q95',
 "change_quantiles{'ql': 0.6, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}",
 'max_to_min_5',
 'mfcc_delta_4_quantile75',
 'abs_max_1',
 '5000crest_factor_quantile75',
 'partial_autocorrelationlag_1',
 'mfcc_delta_11_quantile01',
 'mfcc_accelerate_7_quantile01',
 'mfcc_8_rolling_std_mean',
 'q95_roll_mean_100',
 'mfcc_delta_6_rolling_std_mean',
 'flac3_0_min',
 'fft_coefficientcoeff_36__attr_"abs"',
 'iqr_8',
 'max_9',
 '5000smoothness_quantile05',
 'mfcc_1_kurtosis',
 'mfcc_7_abs_q95',
 'spkt_welch_density__coeff_66',
 'spkt_welch_density__coeff_64',
 'mfcc_3_abs_q95',
 '5000skewness_mean_',
 'mfcc_10_abs_mean',
 '5000quantile99quantile01',
 'mean_change_rate',
 'mfcc_accelerate_5_rolling_std_mean',
 'mfcc_accelerate_5_quantile05',
 '5000rms_median_',
 'flac3_1_abs_mean',
 '5000rms_quantile05',
 '5000quantile75quantile25',
 'mfcc_accelerate_12_min',
 "value_count{'value': -1}",
 'mfcc_12_abs_max',
 'abs_min_5',
 'mfcc_5_abs_q99',
 'mfcc_delta_5_quantile01',
 'mfcc_12_quantile75',
 'fft_coefficientcoeff_56__attr_"imag"',
 'spkt_welch_densitycoeff_5',
 'mfcc_0_abs_q05',
 'mfcc_13_quantile75',
 'mfcc_delta_5_min',
 'mfcc_5_quantile99',
 'fft_coefficientcoeff_8__attr_"angle"',
 'spkt_welch_density__coeff_30',
 'mfcc_accelerate_4_max',
 'mfcc_14_median',
 "change_quantiles{'ql': 0.2, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'}",
 'mfcc_5_median',
 'mfcc_accelerate_5_abs_q75',
 'spkt_welch_density__coeff_59',
 'mfcc_accelerate_5_kurtosis',
 'mfcc_delta_9_quantile99',
 'mfcc_5_quantile25',
 'spkt_welch_density__coeff_113',
 'mfcc_2_quantile05',
 'spkt_welch_density__coeff_58',
 'mfcc_5_abs_q75',
 'spkt_welch_density__coeff_22',
 'spkt_welch_density__coeff_115',
 'spkt_welch_density__coeff_4',
 'mfcc_5_abs_q05',
 'spkt_welch_density__coeff_25',
 'mfcc_5_abs_max',
 'spkt_welch_density__coeff_27',
 'mfcc_4_quantile75',
 'mfcc_3_rolling_std_mean',
 'mfcc_4_median',
 'mfcc_0_quantile01',
 '5000quantile75rssq_',
 'fft_coefficientcoeff_24__attr_"imag"',
 'mfcc_delta_12_kurtosis',
 '5000rms_quantile25',
 'mfcc_13_quantile05',
 'fft_coefficientcoeff_8__attr_"imag"',
 'mfcc_delta_11_rolling_std_mean',
 'mfcc_delta_12_quantile99',
 'mfcc_accelerate_9_abs_q25',
 'mfcc_11_abs_mean',
 'mfcc_delta_15_abs_max',
 'abs_max_2',
 'mfcc_11_abs_std',
 'abs_max_roll_mean_1000',
 'peak_to_average_power_ratio__roll_mean',
 'mfcc_13_median',
 'fft_coefficientcoeff_62__attr_"abs"',
 'mfcc_11_quantile75',
 "quantile{'q': 0.8}",
 'mfcc_delta_3_rolling_std_mean',
 'abs_q99_7',
 'mfcc_delta_12_max',
 'mfcc_12_min',
 'mfcc_10_quantile99',
 'mfcc_14_abs_q99',
 'mfcc_14_quantile05',
 'mfcc_14_quantile25',
 'kurt_1',
 'mfcc_2_median',
 'q01_2',
 'mfcc_2_abs_q25',
 'kurt_8',
 'mfcc_accelerate_6_abs_max',
 'mfcc_1_quantile75',
 'q05_5',
 'abs_q95_9',
 'q05_roll_mean_100',
 'mfcc_10_abs_q25',
 'q75_9',
 'mfcc_15_mean',
 "change_quantiles{'ql': 0.6, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'}",
 'flac3_0_abs_std',
 "change_quantiles{'ql': 0.4, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}"]

In [34]:
db_stacknet_pca_mfcc = DFDB('../trial/stacknet_pca_mfcc.pkl', auto_commit=False)

In [36]:
df_trial = db_stacknet_pca_mfcc.select()

In [43]:
df_trial[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-30 07:07:48.036198,53,1.674009,0.000324,1.936265,0.004385,0.262256,init_try
1,2019-05-30 07:12:03.586210,53,1.665349,0.001921,2.031980,0.008376,0.366632,init_try
2,2019-05-30 07:17:21.403443,53,1.153101,0.000151,2.130778,0.012764,0.977677,tune pca 3
3,2019-05-30 07:17:36.552399,53,1.504163,0.000844,2.016022,0.008231,0.511859,tune pca 3
4,2019-05-30 07:18:22.690953,53,1.162563,0.001045,2.068187,0.007282,0.905624,tune pca 3
5,2019-05-30 07:20:14.247094,53,1.515581,0.000395,1.963594,0.006442,0.448013,tune pca 3
6,2019-05-30 07:21:19.985416,53,1.556997,0.000169,1.972191,0.006742,0.415195,tune pca 3
7,2019-05-30 07:22:05.045309,53,1.141510,0.000892,2.095548,0.008822,0.954038,tune pca 3
8,2019-05-30 07:22:18.175149,53,1.419610,0.000602,2.031881,0.006208,0.612271,tune pca 3
9,2019-05-30 07:22:32.806360,53,1.505584,0.000628,1.987872,0.007841,0.482288,tune pca 3


In [48]:
df_vae_train.columns.tolist(),df_tsne_train.columns.tolist(),

(['kle', 'loss', 'mse', 'reconstructione', 'index', 'z_0', 'z_1'],
 ['index', 'tsne_1', 'tsne_2', 'tsne_3'])

In [53]:
columns356pvae = ['cb452',
  'xgbm1172',
  'lgbm2156',
  'gbm306',
  'rf297',
  'et459',
  'knn17',
  'svr7',
  'fm313',
  'lasso8',
  'ridge15',
  'mlp13',
  'frgf200',
  'spkt_welch_density__coeff_3',
  'abs_q25_5',
  'q25_roll_std_100',
  'abs_q75_6',
  'abs_q75_7',
  'spkt_welch_densitycoeff_2',
  'abs_q01_4',
  'iqr_6',
  'q05_roll_std_100',
  'q05_roll_std_1000',
  'median__roll_std',
  'abs_q01_5',
  "number_peaks{'n': 10}",
  'FFT_Mag_75q0',
  "value_count{'value': 1}",
  'q01_roll_std_100',
  'abs_q95_2',
  'abs_q95_6',
  'MA_1000MA_std_mean_7',
  'q05_roll_std_10',
  'q01_roll_std_1000',
  'abs_max_roll_mean_1000',
  'abs_q75_2',
  'abs_q05_6',
  '5000std_quantile25',
  "number_crossing_m{'m': 1}",
  "autocorrelation{'lag': 5}",
  'q75_roll_std_10',
  'q05_2',
  '5000smoothness_quantile05',
  '5000variance_median__pca6',
  'abs_q05_6_pca1',
  '5000std_quantile75_pca4',
  "range_count{'min': 0, 'max': 1000000000000.0}_pca1",
  'mfcc_5_abs_mean',
  'mfcc_13_mean',
  'mfcc_accelerate_8_variance',
  'mfcc_accelerate_1_kurtosis',
  'mfcc_13_quantile25',
  'mfcc_5_mean','kle', 'loss', 'mse','tsne_1', 'tsne_2']

In [54]:
param = {'columns': columns356pvae,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'num_trees': 397,
   'depth': 6,
   'learning_rate': 0.04568702913200931,
   'l2_leaf_reg': 42.67420651153853,
   'bagging_temperature': 0.8288030970057758,
   'random_strength': 0.648173282450518,
   'random_state': 9734,
   'logging_level': 'Silent'},
  'fit': {}}}

In [55]:
mytrial =[]
EP.select_features_(df_train, param, mytrial, nfeats_best=10, nfeats_removed_per_try=1, key='average_model_weight', remark='rfe to 10 group3')

In [75]:
# mytrial =[]

#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 6)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 0.0001, 1)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_uniform('random_strength', .001, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':df_trial.loc[941]['param']['columns'],
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
                'logging_level': 'Silent'
            },
            'fit':{
            },
        }
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 941 by group')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-06-01 03:18:57,153] Finished a trial resulted in value: 1.8397121072224627. Current best value is 1.8397121072224627 with parameters: {'num_trees': 651, 'depth': 5, 'learning_rate': 0.05872622750804325, 'l2_leaf_reg': 0.327764605397104, 'bagging_temperature': 0.9118788674041807, 'random_strength': 0.8942140385504486, 'random_state': 4179}.
[I 2019-06-01 03:19:24,591] Finished a trial resulted in value: 1.9264913114845246. Current best value is 1.8397121072224627 with parameters: {'num_trees': 651, 'depth': 5, 'learning_rate': 0.05872622750804325, 'l2_leaf_reg': 0.327764605397104, 'bagging_temperature': 0.9118788674041807, 'random_strength': 0.8942140385504486, 'random_state': 4179}.
[I 2019-06-01 03:20:14,897] Finished a trial resulted in value: 1.8320169938726723. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_str

[I 2019-06-01 03:34:04,515] Finished a trial resulted in value: 1.862811283535262. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 03:34:54,855] Finished a trial resulted in value: 1.8358983240728357. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 03:35:27,966] Finished a trial resulted in value: 1.855362434271912. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_str

[I 2019-06-01 03:47:25,847] Finished a trial resulted in value: 1.8754392512181088. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 03:47:50,511] Finished a trial resulted in value: 1.8625533337989342. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 03:48:47,206] Finished a trial resulted in value: 1.8387358669429468. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_s

[I 2019-06-01 04:01:38,164] Finished a trial resulted in value: 1.8421597576410378. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 04:01:57,034] Finished a trial resulted in value: 1.8516952146454064. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 04:02:17,156] Finished a trial resulted in value: 1.8780848912000634. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_s

[I 2019-06-01 04:11:26,752] Finished a trial resulted in value: 1.8442130502453684. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 04:12:10,859] Finished a trial resulted in value: 1.8512861489722248. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 04:12:43,791] Finished a trial resulted in value: 1.8659671654609735. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_s

[I 2019-06-01 04:21:51,585] Finished a trial resulted in value: 1.8372015591091728. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 04:22:18,720] Finished a trial resulted in value: 1.8405200230012095. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 04:22:44,603] Finished a trial resulted in value: 1.8758430412476976. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_s

[I 2019-06-01 04:32:20,062] Finished a trial resulted in value: 1.8444440849527404. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 04:32:34,589] Finished a trial resulted in value: 1.84266775415613. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_strength': 0.6703162699309498, 'random_state': 5122}.
[I 2019-06-01 04:32:59,037] Finished a trial resulted in value: 1.8547628917477947. Current best value is 1.8320169938726723 with parameters: {'num_trees': 592, 'depth': 6, 'learning_rate': 0.04546716839654667, 'l2_leaf_reg': 0.7344106871358916, 'bagging_temperature': 0.9222284679441616, 'random_str

[I 2019-06-01 04:50:19,692] Finished a trial resulted in value: 1.8492147415285947. Current best value is 1.829515205853203 with parameters: {'num_trees': 911, 'depth': 6, 'learning_rate': 0.026743285513422744, 'l2_leaf_reg': 0.8635567708077143, 'bagging_temperature': 0.9716912687901551, 'random_strength': 0.12647142286895785, 'random_state': 3568}.
[I 2019-06-01 04:51:13,189] Finished a trial resulted in value: 1.845041457521171. Current best value is 1.829515205853203 with parameters: {'num_trees': 911, 'depth': 6, 'learning_rate': 0.026743285513422744, 'l2_leaf_reg': 0.8635567708077143, 'bagging_temperature': 0.9716912687901551, 'random_strength': 0.12647142286895785, 'random_state': 3568}.
[I 2019-06-01 04:51:56,406] Finished a trial resulted in value: 1.8400041043625268. Current best value is 1.829515205853203 with parameters: {'num_trees': 911, 'depth': 6, 'learning_rate': 0.026743285513422744, 'l2_leaf_reg': 0.8635567708077143, 'bagging_temperature': 0.9716912687901551, 'random_

[I 2019-06-01 05:06:16,580] Finished a trial resulted in value: 1.8860354728148259. Current best value is 1.829515205853203 with parameters: {'num_trees': 911, 'depth': 6, 'learning_rate': 0.026743285513422744, 'l2_leaf_reg': 0.8635567708077143, 'bagging_temperature': 0.9716912687901551, 'random_strength': 0.12647142286895785, 'random_state': 3568}.
[I 2019-06-01 05:06:50,464] Finished a trial resulted in value: 1.8375415242047257. Current best value is 1.829515205853203 with parameters: {'num_trees': 911, 'depth': 6, 'learning_rate': 0.026743285513422744, 'l2_leaf_reg': 0.8635567708077143, 'bagging_temperature': 0.9716912687901551, 'random_strength': 0.12647142286895785, 'random_state': 3568}.
[I 2019-06-01 05:07:24,019] Finished a trial resulted in value: 1.869025667127463. Current best value is 1.829515205853203 with parameters: {'num_trees': 911, 'depth': 6, 'learning_rate': 0.026743285513422744, 'l2_leaf_reg': 0.8635567708077143, 'bagging_temperature': 0.9716912687901551, 'random_

In [77]:
for trial_i in mytrial:
    db_stacknet_pca_mfcc.insert(trial_i)
df_trial = db_stacknet_pca_mfcc.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])
df_trial['algorithm-init'] = df_trial['param'].apply(lambda x: x['algorithm']['init'])

In [100]:
db_stacknet_pca_mfcc.commit()

In [84]:
df_trial[(df_trial['remark']=='tune 941 by group')&(df_trial['mae_diff']<.1)][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','algorithm-init']].sort_values(by=['val_mae']).head()

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,algorithm-init
1196,2019-06-01 03:37:49.017526,tune 941 by group,15,1.747140,0.000578,1.836753,0.003168,0.089613,"{'num_trees': 699, 'depth': 2, 'learning_rate': 0.0661723981226296, 'l2_leaf_reg': 0.5160804080338326, 'bagging_temperature': 0.7959176162566737, 'random_strength': 0.09984250008140327, 'random_state': 5234, 'logging_level': 'Silent'}"
1198,2019-06-01 03:38:44.365785,tune 941 by group,15,1.745461,0.000527,1.837021,0.003617,0.091560,"{'num_trees': 718, 'depth': 3, 'learning_rate': 0.03120323743802602, 'l2_leaf_reg': 0.1761783064404861, 'bagging_temperature': 0.884063752227235, 'random_strength': 0.38441807259601113, 'random_state': 1589, 'logging_level': 'Silent'}"
1360,2019-06-01 05:11:41.139368,tune 941 by group,15,1.738574,0.000806,1.837050,0.003120,0.098476,"{'num_trees': 837, 'depth': 2, 'learning_rate': 0.06310964316202922, 'l2_leaf_reg': 0.8097822436288219, 'bagging_temperature': 0.7749070018333777, 'random_strength': 0.15220247187154434, 'random_state': 3259, 'logging_level': 'Silent'}"
1250,2019-06-01 04:07:35.796164,tune 941 by group,15,1.752538,0.000590,1.837051,0.003042,0.084512,"{'num_trees': 748, 'depth': 2, 'learning_rate': 0.058205379846642785, 'l2_leaf_reg': 0.4557097378388437, 'bagging_temperature': 0.9636863089955022, 'random_strength': 0.13369582677997985, 'random_state': 8689, 'logging_level': 'Silent'}"
1173,2019-06-01 03:22:32.208930,tune 941 by group,15,1.738392,0.000530,1.837169,0.003141,0.098777,"{'num_trees': 240, 'depth': 3, 'learning_rate': 0.11534789392268938, 'l2_leaf_reg': 0.36438632587262754, 'bagging_temperature': 0.766326926557269, 'random_strength': 0.719822329523175, 'random_state': 6642, 'logging_level': 'Silent'}"


In [61]:
def width_frist_rfe(df_train, param, trial, score, df_test=None, remark=None):

    param_ = copy.deepcopy(param)
    columns_ = param_['columns']
    best_score = score
    best_param = param_
    for col in columns_:
        param_['columns'] = list(set(columns_) - set([col]))
        df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train, param_, df_test=df_test, trial=trial, is_output_feature_importance=False, remark=remark)
        val_mae_mean = np.mean(df_his.valid)
        if val_mae_mean<best_score:
            best_score = val_mae_mean
            best_param = copy.deepcopy(param_)

    if best_score < score:
        width_frist_rfe(df_train, best_param, trial, best_score, df_test, remark=remark)

    return

In [67]:
mytrial = []
width_frist_rfe(df_train, df_trial.loc[762]['param'], mytrial, df_trial.loc[762]['val_mae'], df_test=df_test, remark='wf 762')

In [74]:
df_trial[(df_trial['remark']=='wf 762')][['datetime','kfold','algorithm-init', 'remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].sort_values(by=['val_mae'])

,datetime,kfold,algorithm-init,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
941,2019-06-01 02:55:32.751302,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'num_trees': 397, 'depth': 6, 'learning_rate': 0.04568702913200931, 'l2_leaf_reg': 42.67420651153853, 'bagging_temperature': 0.8288030970057758, 'random_strength': 0.648173282450518, 'random_state': 9734, 'logging_level': 'Silent'}",wf 762,15,1.721383,0.000612,1.835145,0.003192,0.113761
947,2019-06-01 02:58:52.706444,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'num_trees': 397, 'depth': 6, 'learning_rate': 0.04568702913200931, 'l2_leaf_reg': 42.67420651153853, 'bagging_temperature': 0.8288030970057758, 'random_strength': 0.648173282450518, 'random_state': 9734, 'logging_level': 'Silent'}",wf 762,15,1.720992,0.000542,1.835165,0.003596,0.114172
945,2019-06-01 02:57:45.404411,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'num_trees': 397, 'depth': 6, 'learning_rate': 0.04568702913200931, 'l2_leaf_reg': 42.67420651153853, 'bagging_temperature': 0.8288030970057758, 'random_strength': 0.648173282450518, 'random_state': 9734, 'logging_level': 'Silent'}",wf 762,15,1.722317,0.000608,1.835276,0.003835,0.112959
962,2019-06-01 03:06:20.996810,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'num_trees': 397, 'depth': 6, 'learning_rate': 0.04568702913200931, 'l2_leaf_reg': 42.67420651153853, 'bagging_temperature': 0.8288030970057758, 'random_strength': 0.648173282450518, 'random_state': 9734, 'logging_level': 'Silent'}",wf 762,14,1.722167,0.000556,1.835555,0.003123,0.113388
963,2019-06-01 03:06:49.675897,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'num_trees': 397, 'depth': 6, 'learning_rate': 0.04568702913200931, 'l2_leaf_reg': 42.67420651153853, 'bagging_temperature': 0.8288030970057758, 'random_strength': 0.648173282450518, 'random_state': 9734, 'logging_level': 'Silent'}",wf 762,14,1.724545,0.000587,1.835989,0.003161,0.111444
948,2019-06-01 02:59:25.959312,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'num_trees': 397, 'depth': 6, 'learning_rate': 0.04568702913200931, 'l2_leaf_reg': 42.67420651153853, 'bagging_temperature': 0.8288030970057758, 'random_strength': 0.648173282450518, 'random_state': 9734, 'logging_level': 'Silent'}",wf 762,15,1.719641,0.000610,1.836027,0.003834,0.116387
943,2019-06-01 02:56:38.267544,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'num_trees': 397, 'depth': 6, 'learning_rate': 0.04568702913200931, 'l2_leaf_reg': 42.67420651153853, 'bagging_temperature': 0.8288030970057758, 'random_strength': 0.648173282450518, 'random_state': 9734, 'logging_level': 'Silent'}",wf 762,15,1.721473,0.000584,1.836502,0.003870,0.115028
965,2019-06-01 03:07:48.949941,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'num_trees': 397, 'depth': 6, 'learning_rate': 0.04568702913200931, 'l2_leaf_reg': 42.67420651153853, 'bagging_temperature': 0.8288030970057758, 'random_strength': 0.648173282450518, 'random_state': 9734, 'logging_level': 'Silent'}",wf 762,14,1.722849,0.000594,1.836743,0.003357,0.113894
960,2019-06-01 03:05:23.791856,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'num_trees': 397, 'depth': 6, 'learning_rate': 0.04568702913200931, 'l2_leaf_reg': 42.67420651153853, 'bagging_temperature': 0.8288030970057758, 'random_strength': 0.648173282450518, 'random_state': 9734, 'logging_level': 'Silent'}",wf 762,14,1.723275,0.000595,1.836799,0.003561,0.113524
950,2019-06-01 03:00:26.720352,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}","{'num_trees': 397, 'depth': 6, 'learning_rate': 0.04568702913200931, 'l2_leaf_reg': 42.67420651153853, 'bagging_temperature': 0.8288030970057758, 'random_strength': 0.648173282450518, 'random_state': 9734, 'logging_level': 'Silent'}",wf 7

In [83]:
idx=1196
df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission_mystacknet_pca_mfcc_{}.csv'.format(idx), index=False)

In [97]:
df_trial.loc[906]['param']

{'columns': ['mlp13',
  'q01_roll_std_100',
  'mfcc_accelerate_8_variance',
  'abs_q75_2',
  'kle',
  'mfcc_5_abs_mean',
  'xgbm1172',
  'median__roll_std',
  'abs_q05_6_pca1',
  'svr7',
  'ridge15',
  'lgbm2156',
  'spkt_welch_density__coeff_3',
  'gbm306',
  'knn17',
  'lasso8',
  'cb452'],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'num_trees': 397,
   'depth': 6,
   'learning_rate': 0.04568702913200931,
   'l2_leaf_reg': 42.67420651153853,
   'bagging_temperature': 0.8288030970057758,
   'random_strength': 0.648173282450518,
   'random_state': 9734,
   'logging_level': 'Silent'},
  'fit': {}}}

In [ ]:
'mlp13',
'xgbm1172',
'svr7',
'ridge15',
'lgbm2156',
'gbm306',
'knn17',
'lasso8',
'cb452'